In [6]:
from httpx import Client
import os, sys
import json
import pandas as pd
from pprint import pprint

# Move the current working directory to the root of the repository for consistency in file paths
def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")

def pprint_first_row_for_column(df, column_name, i=0):
    print(f"First row for column '{column_name}':")
    pprint(df[column_name].iloc[i])
    return df[column_name].iloc[i]


data = os.path.join("data", "processed", "gtr_links_expanded.csv")
df = pd.read_csv(data)

my_id = pprint_first_row_for_column(df, "id")

# get all rows with id = my_id
df_id = df[df["id"] == my_id]



Current working directory:  c:\Users\dec2g\GitHub\OrgSync
First row for column 'id':
'5331B126-3AB4-4412-B56D-00E8F2796556'


In [7]:
df_id.head()

,id,href_org,links,href_link,rel,start,end
0,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/0D5DF2FF-...,PROJECT,NaN,NaN
1,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/0D0F72CC-...,PROJECT,NaN,NaN
2,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/0C6849FD-...,PROJECT,NaN,NaN
3,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/C4059685-...,PROJECT,NaN,NaN
4,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/F0E04953-...,PROJECT,NaN,NaN


In [8]:
# get all href_links for "rel" = PROJECT
df_id_project = df_id[df_id["rel"] == "PROJECT"]
df_id_project.head()


,id,href_org,links,href_link,rel,start,end
0,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/0D5DF2FF-...,PROJECT,NaN,NaN
1,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/0D0F72CC-...,PROJECT,NaN,NaN
2,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/0C6849FD-...,PROJECT,NaN,NaN
3,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/C4059685-...,PROJECT,NaN,NaN
4,5331B126-3AB4-4412-B56D-00E8F2796556,http://gtr.ukri.org/gtr/api/organisations/5331...,{'href': 'http://gtr.ukri.org/gtr/api/projects...,http://gtr.ukri.org/gtr/api/projects/F0E04953-...,PROJECT,NaN,NaN


In [9]:
links = df_id_project["href_link"].to_list()
print(links)

['http://gtr.ukri.org/gtr/api/projects/0D5DF2FF-B732-4218-B0E3-4FFBF3DDC906', 'http://gtr.ukri.org/gtr/api/projects/0D0F72CC-0163-47CE-A462-5FDDBA4C1C38', 'http://gtr.ukri.org/gtr/api/projects/0C6849FD-CA2D-49A9-80D4-75F710980208', 'http://gtr.ukri.org/gtr/api/projects/C4059685-9263-44E6-B89A-D8609FCC1360', 'http://gtr.ukri.org/gtr/api/projects/F0E04953-58F2-46FB-9837-3C341A3D3165', 'http://gtr.ukri.org/gtr/api/projects/D9A401D9-6D91-4DA6-9012-7137774B1DD8', 'http://gtr.ukri.org/gtr/api/projects/EC918CE3-4862-4EED-884A-E1AC42C8F46D', 'http://gtr.ukri.org/gtr/api/projects/ADB18319-E0B8-4735-AC8B-59946A4BB9CB', 'http://gtr.ukri.org/gtr/api/projects/DCFE5B2E-2EC2-4AA2-9244-EE252CED60E4', 'http://gtr.ukri.org/gtr/api/projects/1E0BB71C-795B-4D20-B46C-2E618CAC96C0', 'http://gtr.ukri.org/gtr/api/projects/2294E699-BBAC-47CB-9293-6BAC6BED2640', 'http://gtr.ukri.org/gtr/api/projects/A45E2042-8FFF-40EF-B270-15D95481946E', 'http://gtr.ukri.org/gtr/api/projects/096F035D-FAEB-4187-9C63-1DA5337E85B9'

In [11]:
def get_end_of_link(link):
    # convert links like 'http://gtr.ukri.org/gtr/api/projects/E0B948B2-4771-4063-BCF0-2E0B58080049', 
    # to 'E0B948B2-4771-4063-BCF0-2E0B58080049'
    return link.split("/")[-1]
print(get_end_of_link(links[0]))

0D5DF2FF-B732-4218-B0E3-4FFBF3DDC906


Use guide https://gtr.ukri.org/resources/GtR-2-API-v1.7.4.pdf

and api_v2_scraper_outcomes.ipynb

Can use xlm or json in headers{"accept": ...}
* application/vnd.rcuk.gtr.xml-v7
* application/vnd.rcuk.gtr.json-v7

In [12]:
httpx = Client(
    headers={"accept": "application/vnd.rcuk.gtr.json-v7"},
    base_url="http://gtr.ukri.org/gtr/api/projects",
    follow_redirects=True,
    timeout=30,
)

domains =[
    {"url": "publications", "key": "publication"},
]
ENDPOINT = domains[0]


for link in links:
    link = get_end_of_link(link)
    result = httpx.get(link).json()
    pprint(result)



{'abstractText': 'Abstracts are not currently available in GtR for all funded '
                 'research. This is normally because the abstract was not '
                 'required at the time of proposal submission, but may be '
                 'because it included sensitive information such as personal '
                 'details.',
 'created': 1720391293000,
 'end': None,
 'ext': None,
 'grantCategory': 'Research Grant',
 'healthCategories': {'healthCategory': []},
 'href': 'http://gtr.ukri.org/gtr/api/projects/0D5DF2FF-B732-4218-B0E3-4FFBF3DDC906',
 'id': '0D5DF2FF-B732-4218-B0E3-4FFBF3DDC906',
 'identifiers': {'identifier': [{'type': 'RCUK', 'value': 'ES/I028072/1'}]},
 'leadFunder': 'ESRC',
 'leadOrganisationDepartment': 'Business School',
 'links': {'link': [{'end': None,
                     'href': 'http://gtr.ukri.org/gtr/api/persons/42FDE65C-3EDB-4494-AD0A-F31125B8C0A0',
                     'otherAttributes': {},
                     'rel': 'PI_PER',
                    

next, go to publication link and get author, or check for authors in the project link

In [14]:
result = httpx.get("0D5DF2FF-B732-4218-B0E3-4FFBF3DDC906").json()

In [15]:
result

{'links': {'link': [{'href': 'http://gtr.ukri.org/gtr/api/persons/42FDE65C-3EDB-4494-AD0A-F31125B8C0A0',
    'rel': 'PI_PER',
    'start': None,
    'end': None,
    'otherAttributes': {}},
   {'href': 'http://gtr.ukri.org/gtr/api/persons/9E092DB5-1688-4C96-988D-FE9945730307',
    'rel': 'COI_PER',
    'start': None,
    'end': None,
    'otherAttributes': {}},
   {'href': 'http://gtr.ukri.org/gtr/api/organisations/AC11EAE6-D32A-4C68-B6BB-8059BE64FEF1',
    'rel': 'LEAD_ORG',
    'start': None,
    'end': None,
    'otherAttributes': {}},
   {'href': 'http://gtr.ukri.org/gtr/api/organisations/5331B126-3AB4-4412-B56D-00E8F2796556',
    'rel': 'COLLAB_ORG',
    'start': None,
    'end': None,
    'otherAttributes': {}},
   {'href': 'http://gtr.ukri.org/gtr/api/organisations/65DB5820-053E-436C-856D-86976BB40EE2',
    'rel': 'COLLAB_ORG',
    'start': None,
    'end': None,
    'otherAttributes': {}},
   {'href': 'http://gtr.ukri.org/gtr/api/funds/8963E2CF-21E3-42BC-9068-8DB325F62AA5',
   